# Location

- Merge 2016 and 2022 datasets for location

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
from pprint import pprint

elec_2022 = pd.read_csv("../dataset/Elections Dataset/pres_precinct_data_2022.csv")
elec_2019 = pd.read_json("../dataset/Elections Dataset/2019/philippine_provinces_cities_municipalities_and_barangays_2019v2.json")
elec_2016 = pd.read_csv("../dataset/Elections Dataset/vp_precinct_data_2016.csv")

## Rename

In [2]:
TO_LOC_2022_COLUMNS = {
    'barangay': 'Barangay', 
    'province': 'Province', 
    'region': 'Region', 
    'municipality': 'City', 
    'clustered_precinct': 'Precinct ID (2016)'
}
elec_2016.rename(columns=TO_LOC_2022_COLUMNS, inplace=True)
elec_2022.rename(columns={'Precinct ID': 'Precinct ID (2022)'}, inplace=True)

## Select locations subset w/ Precinct ID

In [3]:
columns_2016 = ['Precinct ID (2016)', 'Barangay', 'Province', 'Region', 'City']
columns_2022 = ['Precinct ID (2022)', 'Barangay', 'Province', 'Region', 'City']
loc_2016 = pd.DataFrame(elec_2016[columns_2016].sort_index(axis=1).reset_index(drop=True))
loc_2022 = pd.DataFrame(elec_2022[columns_2022].sort_index(axis=1).reset_index(drop=True))

## Abbreviate regions in 2022 dataset

In [4]:
# https://stackoverflow.com/questions/19937362/filter-string-data-based-on-its-string-length
loc_2022.loc[
    loc_2022["Region"].apply(lambda name: len(name.split()) > 2), "Region"
] = loc_2022["Region"].apply(lambda name: "".join([word[0] for word in name.split()]))

## Prepare dataframes for merging

In [5]:
loc_2016.sort_values(by='Precinct ID (2016)', inplace=True)
loc_2022.sort_values(by='Precinct ID (2022)', inplace=True)

# https://stackoverflow.com/questions/40251948/stop-pandas-from-converting-int-to-float-due-to-an-insertion-in-another-column
loc_2016["Precinct ID (2016)"] = pd.Series(loc_2016["Precinct ID (2016)"], dtype=object)
loc_2022["Precinct ID (2022)"] = pd.Series(loc_2022["Precinct ID (2022)"], dtype=object) 

# https://stackoverflow.com/questions/33165734/update-index-after-sorting-data-frame
loc_2016 = loc_2016.reset_index(drop=True)
loc_2022 = loc_2022.reset_index(drop=True)

## Check matched dataframes

In [6]:
print('Shape', loc_2016.shape, loc_2022.shape)
print(loc_2016.columns)
print(loc_2022.columns)

Shape (90642, 5) (105666, 5)
Index(['Barangay', 'City', 'Precinct ID (2016)', 'Province', 'Region'], dtype='object')
Index(['Barangay', 'City', 'Precinct ID (2022)', 'Province', 'Region'], dtype='object')


## Merge the dataframes

In [28]:
merged = pd.merge(
    loc_2016, 
    loc_2022, 
    how="outer",
    left_on=['Precinct ID (2016)'],
    right_on=['Precinct ID (2022)'],
    suffixes=(' (2016)', ' (2022)'),
)

sorter = [
    'Precinct ID (2016)',
    'Precinct ID (2022)',
    'Barangay (2016)',
    'Barangay (2022)',
    'City (2016)',
    'City (2022)',
    'Province (2016)',
    'Province (2022)',
    'Region (2016)',
    'Region (2022)',
]
merged = merged[sorter]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## VALIDATE by comparing to pres and vp pandas profiling reports

In [ ]:
profile = ProfileReport(merged, title="Pandas Profiling Report")
profile.to_file("../reports/Loc - Pandas Profiling Report.html")

## Compare 2016 and 2022 records for matched precinct ids but unmatched location names

In [27]:
locs = ['Barangay', 'Province', 'Region', 'City']
merged.loc[
    (merged["Precinct ID (2016)"] == merged["Precinct ID (2022)"]) & (merged["Region (2016)"] != merged["Region (2022)"]), 
    ["Region (2016)", "Region (2022)"]
]

,Region (2016),Region (2022)
4206,ARMM,BARMM
4207,ARMM,BARMM
4208,ARMM,BARMM
4210,ARMM,BARMM
4211,ARMM,BARMM
...,...,...
76273,ARMM,BARMM
76274,ARMM,BARMM
76275,ARMM,BARMM
76276,ARMM,BARMM


## Check merged dataframe

In [29]:
print(merged.columns)

Index(['Precinct ID (2016)', 'Precinct ID (2022)', 'Barangay (2016)',
       'Barangay (2022)', 'City (2016)', 'City (2022)', 'Province (2016)',
       'Province (2022)', 'Region (2016)', 'Region (2022)'],
      dtype='object')


## Select new precincts 

In [389]:
merged.loc[merged["Barangay (2016)"].isnull()]

,Precinct ID (2016),Precinct ID (2022),Barangay (2016),Barangay (2022),City (2016),City (2022),Province (2016),Province (2022),Region (2016),Region (2022)
90642,NaN,1010048,NaN,SAGAP,NaN,BANGUED,NaN,ABRA,NaN,CAR
90643,NaN,1010049,NaN,SAN ANTONIO,NaN,BANGUED,NaN,ABRA,NaN,CAR
90644,NaN,1010050,NaN,SAN ANTONIO,NaN,BANGUED,NaN,ABRA,NaN,CAR
90645,NaN,1010051,NaN,SAO-ATAN,NaN,BANGUED,NaN,ABRA,NaN,CAR
90646,NaN,1010052,NaN,SAPPAAC,NaN,BANGUED,NaN,ABRA,NaN,CAR
...,...,...,...,...,...,...,...,...,...,...
108029,NaN,93100021,NaN,BARCELONA PCG,NaN,KINGDOM OF SPAIN,NaN,EUROPE,NaN,OAV
108030,NaN,93100022,NaN,BARCELONA PCG,NaN,KINGDOM OF SPAIN,NaN,EUROPE,NaN,OAV
108031,NaN,93100023,NaN,BARCELONA PCG,NaN,KINGDOM OF SPAIN,NaN,EUROPE,NaN,OAV
108032,NaN,93100026,NaN,BARCELONA PCG,NaN,KINGDOM OF SPAIN,NaN,EUROPE,NaN,OAV


## [DOING] Define function to get dataframe comparison between 2016 to 2022 for a given location

In [35]:
# DOING 
def get_distinct_locs_df(loc):
    """ Get unique values from given {loc} for years 2016 and 2022 """
    locs = {}
    locs[f"Distinct {loc} (2016)"] = merged[f"{loc} (2016)"].dropna().explode().unique()
    locs[f"Distinct {loc} (2016)"] = pd.Series(
        locs[f"Distinct {loc} (2016)"], 
        index=[i for i in range(len(locs[f"Distinct {loc} (2016)"]))], 
        name=f"{loc} (2016)"
    )
    locs[f"Distinct {loc} (2022)"] = merged[f"{loc} (2022)"].dropna().explode().unique()
    locs[f"Distinct {loc} (2022)"] = pd.Series(
        locs[f"Distinct {loc} (2022)"], 
        index=[i for i in range(len(locs[f"Distinct {loc} (2022)"]))], 
        name=f"{loc} (2022)"
    )

    # DOING: account for instanaces such as:
    # ZONE 1 POB. (NALASIN) -vs- ZONE 1 POB.
    
    distinct_provinces = pd.merge(
        locs[f"Distinct {loc} (2016)"], 
        locs[f"Distinct {loc} (2022)"], 
        how="outer",
        left_on=[f'{loc} (2016)'],
        right_on=[f'{loc} (2022)'],
    )
    return distinct_provinces

def get_distinct_locs_df(loc):
    """ Get unique values from given {loc} for years 2016 and 2022 """
    locs = {}
    locs[f"Distinct {loc} (2016)"] = merged.loc[merged["Precinct ID (2016)"].notnull(), f"{loc} (2016)"].dropna().explode().unique()
    locs[f"Distinct {loc} (2016)"] = pd.Series(
        locs[f"Distinct {loc} (2016)"], 
        index=[i for i in range(len(locs[f"Distinct {loc} (2016)"]))], 
        name=f"{loc} (2016)"
    )
    locs[f"Distinct {loc} (2022)"] = merged.loc[merged["Precinct ID (2022)"].notnull(), f"{loc} (2022)"].dropna().explode().unique()
    locs[f"Distinct {loc} (2022)"] = pd.Series(
        locs[f"Distinct {loc} (2022)"], 
        index=[i for i in range(len(locs[f"Distinct {loc} (2022)"]))], 
        name=f"{loc} (2022)"
    )
    if locs[f"Distinct {loc} (2016)"].count() > locs[f"Distinct {loc} (2022)"].count(): 
        locs[f"Distinct {loc} (2022)"] = locs[f"Distinct {loc} (2022)"].reindex_like(locs[f"Distinct {loc} (2016)"])
        # print(locs[f"Distinct {loc} (2022)"])
        # print(locs[f"Distinct {loc} (2016)"])
    else:
        locs[f"Distinct {loc} (2016)"] = locs[f"Distinct {loc} (2016)"].reindex_like(locs[f"Distinct {loc} (2022)"])

    return pd.concat([locs[f"Distinct {loc} (2016)"], locs[f"Distinct {loc} (2022)"]], axis=1)

In [36]:
# compare distinct regions from 2016 and 2022

distinct_precincts = get_distinct_locs_df("Precinct ID")
# ProfileReport(distinct_precincts)

In [37]:
# compare distinct regions from 2016 and 2022

distinct_regions = get_distinct_locs_df("Region")
# ProfileReport(distinct_regions)

In [38]:
# compare distinct provinces from 2016 and 2022

distinct_provinces = get_distinct_locs_df("Province")
# ProfileReport(distinct_provinces)

In [39]:
# compare distinct cities from 2016 and 2022

distinct_cities = get_distinct_locs_df("City")
# ProfileReport(distinct_cities)

In [40]:
# compare distinct barangays from 2016 and 2022

distinct_barangays = get_distinct_locs_df("Barangay")
# ProfileReport(distinct_barangays)

## [DOING] Define functions to get new and old locations

In [41]:
loc_columns = {
    'Precinct ID': distinct_precincts, 
    'Barangay': distinct_barangays, 
    'City': distinct_cities,
    'Province': distinct_provinces, 
    'Region': distinct_regions, 
}

def get_added_and_removed():
    for loc_col in loc_columns.keys():
        print(
            loc_columns[loc_col]
            .loc[loc_columns[loc_col][f"{loc_col} (2016)"].isnull(), f"{loc_col} (2022)"],
            end="\n\n\n"
        )
        print(
            loc_columns[loc_col]
            .loc[loc_columns[loc_col][f"{loc_col} (2022)"].isnull(), f"{loc_col} (2016)"],
            end="\n\n\n"
        )

In [42]:
loc = 'Barangay'
locs = {}
locs[f"Distinct {loc} (2016)"] = merged.loc[merged["Precinct ID (2016)"].notnull(), f"{loc} (2016)"].dropna().explode().unique()
# list(locs[f"Distinct {loc} (2016)"])